In [1]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=730)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('BTC-USD',
                      start=start_date,
                      end=end_date,
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)
print(data.head())

        Date          Open          High           Low         Close  \
0 2021-07-21  29796.285156  32752.326172  29526.183594  32110.693359   
1 2021-07-22  32138.873047  32576.400391  31745.298828  32313.105469   
2 2021-07-23  32305.958984  33581.550781  32057.892578  33581.550781   
3 2021-07-24  33593.730469  34490.390625  33424.859375  34292.445312   
4 2021-07-25  34290.292969  35364.925781  33881.835938  35350.187500   

      Adj Close       Volume  
0  32110.693359  28203024559  
1  32313.105469  19555230518  
2  33581.550781  22552046192  
3  34292.445312  21664706865  
4  35350.187500  20856685287  


In [2]:
data.shape

(729, 7)

In [3]:
#let’s visualize the change in bitcoin prices till today by using a candlestick chart:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"],
                                        high=data["High"],
                                        low=data["Low"],
                                        close=data["Close"])])
figure.update_layout(title = "Bitcoin Price Analysis",
                     xaxis_rangeslider_visible=False)
figure.show()

In [4]:
#So, let’s have a look at the correlation of all the columns in the data concerning the Close column:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Adj Close    1.000000
High         0.998268
Low          0.998193
Open         0.996050
Volume       0.199528
Name: Close, dtype: float64


<ipython-input-4-ac73ebc5cdca>:2: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [ ]:
#I will be using the AutoTS library to predict the bitcoin prices for the next 30 days:
from autots import AutoTS
model = AutoTS(forecast_length=30, frequency='infer', ensemble='simple')
model = model.fit(data, date_col='Date', value_col='Close', id_col=None)
prediction = model.predict()
forecast = prediction.forecast
print(forecast)